In [1]:
import pandas as pd
dados_reviews = pd.read_csv('dados_reviews_tratados.csv', sep = ',')
dados_reviews['content'] = dados_reviews['content'].fillna('')
dados_reviews = dados_reviews[~dados_reviews['sentiment'].isin(['surprise', 'fear'])]

# treinamento e teste

Aqui vamos estimar a performance do classificador utilizando uma subdivisão nos dados. Vamos dividir os dados (BOW_dataframe) em um conjunto de teste e outro treinamento. 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from gensim.models import Word2Vec

# Carregar os dados
dados_reviews = pd.read_csv('dados_reviews_tratados.csv', sep=',')
dados_reviews['content'] = dados_reviews['content'].fillna('')
dados_reviews = dados_reviews[~dados_reviews['sentiment'].isin(['surprise', 'fear','neutral'])]

# Treinamento do modelo Word2Vec
sentences = [sentence.split() for sentence in dados_reviews['content']]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Função para obter a média dos vetores das palavras
def get_mean_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Aplicar a função para todas as reviews
X = np.array([get_mean_vector(sentence.split(), model) for sentence in dados_reviews['content']])
y = np.array(dados_reviews['sentiment'])

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo de classificação
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Previsão e avaliação
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Novo DataFrame com reviews para teste
teste_emocoes = pd.read_csv('teste.csv', sep=',', encoding='latin-1')

# Função de tokenização
def tokenize(text):
    return text.lower().split()

# Função para prever o sentimento de um review
def predict_sentiment(review):
    tokens = tokenize(review)
    review_vector = get_mean_vector(tokens, model)
    review_vector = review_vector.reshape(1, -1)  # Transformar em formato 2D
    return clf.predict(review_vector)[0]  # Retornar a previsão

# Aplicar a função de previsão a cada review no DataFrame
teste_emocoes['sentimento_previsto'] = teste_emocoes['content'].apply(predict_sentiment)

print('emocao predita:')
print(list(teste_emocoes['sentimento_previsto']))
print('emocao real:')
print(list(teste_emocoes['sentiment']))


              precision    recall  f1-score   support

       anger       0.36      0.19      0.25       163
     disgust       0.41      0.58      0.48       169
   happiness       0.40      0.24      0.30        58
     sadness       0.58      0.67      0.62       186

    accuracy                           0.47       576
   macro avg       0.44      0.42      0.41       576
weighted avg       0.45      0.47      0.44       576

emocao predita:
['anger', 'disgust', 'happiness', 'disgust', 'sadness', 'happiness', 'disgust', 'sadness', 'happiness']
emocao real:
['disgust', 'sadness', 'happiness', 'happiness', 'anger', 'sadness', 'anger', 'happiness', 'happiness']
